***GENERATED CODE FOR prep3 PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

***OPERATION FUNCTIONS***

In [ ]:


import pyspark
from dask.dataframe import from_pandas
import json


def calculateDateDifference(df, functionsData, applyOn):
    for obj in functionsData:
        if obj['outputColumn'] == "newColumn":
            ouputColumnName = obj['newColumnName']
        else:
            ouputColumnName = obj['outputColumn']
        columnA = obj['ColumnA']
        columnB = obj['ColumnB']
        df[ouputColumnName] = (df[columnA] - df[columnB]).dt.days
    return df


def runDataCleansing(sparkDf, spark, config):
    configObj = json.loads(config)
    sparkDf.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = from_pandas((sparkDf.toPandas()), npartitions=5)
    functionList = configObj['functionsApplied']
    Data_Cleansing_Methods = {"replaceBy": replaceValues,
                              "formula": calculateFormula,
                              "aggregate": aggregation,
                              "converttostringtype": changeToString,
                              "editname": renameColumns}
    for function in functionList:
        function['functionName']
        df = Data_Cleansing_Methods[function['functionName']](df, function['functionsData'],
                                                              function['applyOn'])
    sparkDf = spark.createDataFrame(df.compute())

    display(sparkDf.limit(2).toPandas())
    return sparkDf


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.functions import col, when
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'Timestamp', 'transformation_label': 'Extract Date'}], 'feature': 'Timestamp', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
                                                'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'Timestamp'}, {'feature_label': 'Timestamp', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('Timestamp')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
#%run prep3Hooks.ipynb
try:
	#sourcePreExecutionHook()

	movieratingstrain = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/MovieRatingsTrain.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line', 'filename': 'MovieRatingsTrain.csv', 'results_url': 'http://localhost:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***PERFORMING OPERATIONS***

In [ ]:
#%run prep3Hooks.ipynb
try:
	#operationPreExecutionHook()

datapreparation = runDataCleansing(movieratingstrain,spark,json.dumps( {"url": "/FileStore/platform/uploadedSourceFiles/MovieRatingsTrain.csv", "source_attributes": {}, "DataPrepFile": "/FileStore/platform/uploadedSourceFiles/MovieRatingsTrain.csv", "data_source": "localfiles", "startListenerOnly": 1, "dateColumnNames": ["Timestamp", "Timestampss"], "FilePath": "/FileStore/platform/extra/6091baca17fc768edfea15ad1620165206/0part.csv", "requestRatio": 50.0, "totalRows": 25000, "BasicStats": {"missingValues": 0.0, "numberOfColumns": 6, "numberOfRows": 25000, "duplicateRowCount": 0, "stats": [{"column": "UserId", "alias": "UserId", "generated": 0, "type": "real", "max": 943.0, "min": 1.0, "mean": 462.82744, "missing": 0.0, "stddev": 267.09}, {"column": "MovieId", "alias": "MovieId", "generated": 0, "type": "real", "max": 1682.0, "min": 1.0, "mean": 425.34048, "missing": 0.0, "stddev": 330.71}, {"column": "Rating", "alias": "Rating", "generated": 0, "type": "real", "max": 5.0, "min": 1.0, "mean": 3.53164, "missing": 0.0, "stddev": 1.12}, {"column": "Timestamp", "alias": "Timestamp", "generated": 0, "type": "date", "max": "1998124", "min": "1997110", "mean": "", "missing": 0.0, "stddev": ""}, {"column": "AvgRating", "alias": "AvgRating", "generated": 0, "type": "real", "max": 4.94, "min": 1.51, "mean": 3.5341776, "missing": 0.0, "stddev": 0.46}, {"column": "Timestampss", "alias": "Timestampss", "generated": 0, "type": "numeric", "max": 0, "min": 0, "mean": 0.0, "missing": 0.0, "stddev": 0.0}]}, "HasBasicStats": 1, "functionsApplied": [{"functionName": "dateDifference", "applyOn": [{"columnName": "Timestamp", "type": "date", "min": "1997110", "max": "1998124", "mean": "-"}, {"columnName": "Timestampss", "type": "date", "min": "1997110", "max": "1998124", "mean": "-"}], "functionsData": [{"newColumnName": "", "outputColumn": "Timestampss", "ColumnA": "Timestamp", "ColumnB": "Timestampss"}]}], "functionChanges": [{"columnName": "Timestamp", "functionName": "Date Difference", "Type": "date", "Parameters": [{"newColumnName": "", "outputColumn": "Timestampss", "ColumnA": "Timestamp", "ColumnB": "Timestampss"}]}, {"columnName": "Timestampss", "functionName": "Date Difference", "Type": "date", "Parameters": [{"newColumnName": "", "outputColumn": "Timestampss", "ColumnA": "Timestamp", "ColumnB": "Timestampss"}]}], "fileheader": [{"field": "UserId", "alias": "UserId", "generated": 0, "position": 1, "type": "real"}, {"field": "MovieId", "alias": "MovieId", "generated": 0, "position": 2, "type": "real"}, {"field": "Rating", "alias": "Rating", "generated": 0, "position": 3, "type": "real"}, {"field": "Timestamp", "alias": "Timestamp", "generated": 0, "position": 4, "type": "date"}, {"field": "AvgRating", "alias": "AvgRating", "generated": 0, "position": 5, "type": "real"}, {"field": "Timestampss", "alias": "Timestampss", "generated": 0, "position": 6, "type": "numeric"}]}))
	#operationPostExecutionHook(datapreparation)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run prep3Hooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(datapreparation,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "UserId", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "467.57", "stddev": "248.41", "min": "2.0", "max": "938.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "UserId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "MovieId", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "478.96", "stddev": "369.18", "min": "1.0", "max": "1648.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "MovieId"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Rating", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.41", "stddev": "1.18", "min": "1.0", "max": "5.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Rating"}, {"transformationsData": [{"feature_label": "Timestamp", "transformation_label": "Extract Date"}], "feature": "Timestamp", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "Timestamp"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "AvgRating", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "3.47", "stddev": "0.46", "min": "1.8", "max": "4.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "AvgRating"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Timestampss", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.0", "stddev": "0.0", "min": "0", "max": "0", "missing": "0"}, "updatedLabel": "Timestampss"}, {"feature": "Timestamp_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "16.29", "stddev": "8.61", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "Timestamp_dayofmonth"}, {"feature": "Timestamp_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "7.25", "stddev": "4.18", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "Timestamp_month"}, {"feature": "Timestamp_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "1997.42", "stddev": "0.49", "min": "1997", "max": "1998", "missing": "0"}, "updatedLabel": "Timestamp_year"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run prep3Hooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["UserId", "MovieId", "Rating", "Timestampss", "Timestamp_dayofmonth", "Timestamp_month", "Timestamp_year"], "AvgRating")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

